# Задача 1.Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру. Mobile Games AB Testing with Cookie Cats

In [204]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')


file_name = './data/AB_Test_Results_2.xlsx'
xl_data = pd.ExcelFile(file_name)
df = xl_data.parse('cookie_cats 2')
df['userid'] = df['userid'].astype(int)
df['sum_gamerounds'] = df['sum_gamerounds'].astype(int)
df['retention_1'] = df['retention_1'].astype(int)
df['retention_7'] = df['retention_7'].astype(int)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


Типы данных корректны. В строках отсутствуют нулевые значения (Null). Проверим разделение групп.

In [205]:
df = pd.pivot_table(df, index='userid', columns='version',
                    values=['sum_gamerounds', 'retention_1', 'retention_7',], aggfunc=np.sum)
df.head(10)


retention_1         retention_7         sum_gamerounds        
version     gate_30 gate_40     gate_30 gate_40        gate_30 gate_40
userid                                                                
116             0.0     NaN         0.0     NaN            3.0     NaN
337             1.0     NaN         0.0     NaN           38.0     NaN
377             NaN     1.0         NaN     0.0            NaN   165.0
483             NaN     0.0         NaN     0.0            NaN     1.0
488             NaN     1.0         NaN     1.0            NaN   179.0
540             NaN     1.0         NaN     1.0            NaN   187.0
1066            0.0     NaN         0.0     NaN            0.0     NaN
1444            NaN     0.0         NaN     0.0            NaN     2.0
1574            NaN     1.0         NaN     1.0            NaN   108.0
1587            NaN     1.0         NaN     0.0            NaN   153.0

In [206]:
df[(df.retention_1.gate_30.notnull()) & (
    df.retention_1.gate_40.notnull())].describe()


retention_1         retention_7         sum_gamerounds        
version     gate_30 gate_40     gate_30 gate_40        gate_30 gate_40
count           0.0     0.0         0.0     0.0            0.0     0.0
mean            NaN     NaN         NaN     NaN            NaN     NaN
std             NaN     NaN         NaN     NaN            NaN     NaN
min             NaN     NaN         NaN     NaN            NaN     NaN
25%             NaN     NaN         NaN     NaN            NaN     NaN
50%             NaN     NaN         NaN     NaN            NaN     NaN
75%             NaN     NaN         NaN     NaN            NaN     NaN
max             NaN     NaN         NaN     NaN            NaN     NaN

In [207]:
df[(df.retention_7.gate_30.notnull()) & (
    df.retention_7.gate_40.notnull())].describe()


retention_1         retention_7         sum_gamerounds        
version     gate_30 gate_40     gate_30 gate_40        gate_30 gate_40
count           0.0     0.0         0.0     0.0            0.0     0.0
mean            NaN     NaN         NaN     NaN            NaN     NaN
std             NaN     NaN         NaN     NaN            NaN     NaN
min             NaN     NaN         NaN     NaN            NaN     NaN
25%             NaN     NaN         NaN     NaN            NaN     NaN
50%             NaN     NaN         NaN     NaN            NaN     NaN
75%             NaN     NaN         NaN     NaN            NaN     NaN
max             NaN     NaN         NaN     NaN            NaN     NaN

In [208]:
df[(df.sum_gamerounds.gate_30.notnull()) & (
    df.sum_gamerounds.gate_40.notnull())].describe()


retention_1         retention_7         sum_gamerounds        
version     gate_30 gate_40     gate_30 gate_40        gate_30 gate_40
count           0.0     0.0         0.0     0.0            0.0     0.0
mean            NaN     NaN         NaN     NaN            NaN     NaN
std             NaN     NaN         NaN     NaN            NaN     NaN
min             NaN     NaN         NaN     NaN            NaN     NaN
25%             NaN     NaN         NaN     NaN            NaN     NaN
50%             NaN     NaN         NaN     NaN            NaN     NaN
75%             NaN     NaN         NaN     NaN            NaN     NaN
max             NaN     NaN         NaN     NaN            NaN     NaN

In [209]:
df[(df.retention_1.gate_30 > 1) | (df.retention_1.gate_40 > 1)].describe()


retention_1         retention_7         sum_gamerounds        
version     gate_30 gate_40     gate_30 gate_40        gate_30 gate_40
count           0.0     0.0         0.0     0.0            0.0     0.0
mean            NaN     NaN         NaN     NaN            NaN     NaN
std             NaN     NaN         NaN     NaN            NaN     NaN
min             NaN     NaN         NaN     NaN            NaN     NaN
25%             NaN     NaN         NaN     NaN            NaN     NaN
50%             NaN     NaN         NaN     NaN            NaN     NaN
75%             NaN     NaN         NaN     NaN            NaN     NaN
max             NaN     NaN         NaN     NaN            NaN     NaN

Группы разделены корректно — идентификаторы пользователей в группах не пересекаются. Идентификаторы пользователей не дублируются. Датасет корректный.

Конверсия не распределяется по нормальному закону, равно как и количество сыгранных раундов. Сформируем выборки и проверим нормальность распределения количества сыгранных раундов.

In [210]:
retention_1_g30 = df[(df.retention_1.gate_30.notnull())
                     ]['retention_1']['gate_30']
retention_7_g30 = df[(df.retention_7.gate_30.notnull())
                     ]['retention_7']['gate_30']
game_rounds_g30 = df[(df.sum_gamerounds.gate_30.notnull())
                     ]['sum_gamerounds']['gate_30']
retention_1_g40 = df[(df.retention_1.gate_40.notnull())
                     ]['retention_1']['gate_40']
retention_7_g40 = df[(df.retention_7.gate_40.notnull())
                     ]['retention_7']['gate_40']
game_rounds_g40 = df[(df.sum_gamerounds.gate_40.notnull())
                     ]['sum_gamerounds']['gate_40']

print(f'pvalue_1 = {stats.shapiro(game_rounds_g30)[1]}')
print(f'pvalue_1 = {stats.shapiro(game_rounds_g40)[1]}')


pvalue_1 = 0.0
pvalue_1 = 0.0


Равенство дисперсий для сыгранных раундов проверим с помощью теста Левена.

In [211]:

print(f'pvalue_1 = {stats.levene(game_rounds_g30, game_rounds_g40)[1]}')


pvalue_1 = 0.46694516772623273


Выборки сыгранных раундов не подчиняется нормальному закону распределения (p-value = 0), дисперсии равны.

Рассчитаем конверсию в вернувшегося пользователя на следующий день и через неделю для наших групп и рассчитаем доверительный интервал:

In [212]:
conv_1_30 = retention_1_g30.sum() / retention_1_g30.count()
conv_1_40 = retention_1_g40.sum() / retention_1_g40.count()
conv_7_30 = retention_7_g30.sum() / retention_7_g30.count()
conv_7_40 = retention_7_g40.sum() / retention_7_g40.count()

print(conv_1_30)
print(conv_1_40)
n_1 = retention_1_g30.count()
n_2 = retention_1_g40.count()
z_score = 1.96
x_mean = conv_1_30 - conv_1_40
delta = z_score * np.sqrt(conv_1_30 * (1 - conv_1_30) /
                          n_1 + conv_1_40 * (1 - conv_1_40) / n_2)
print(f'[{round((x_mean - delta) * 100, 2)} %, {round((x_mean + delta) * 100, 2)} %]')

print(conv_7_30)
print(conv_7_40)
z_score = 1.96
x_mean = conv_7_30 - conv_7_40
delta = z_score * np.sqrt(conv_7_30 * (1 - conv_7_30) /
                          n_1 + conv_7_40 * (1 - conv_7_40) / n_2)
print(f'[{round((x_mean - delta) * 100, 2)} %, {round((x_mean + delta) * 100, 2)} %]')


0.4481879194630872
0.44228274967574577
[-0.06 %, 1.24 %]
0.19020134228187918
0.18200004396667327
[0.31 %, 1.33 %]


Конверсия в пользователя, который вернулся в игру через 1 день после установки для gate_30 и gate_40 не имеет статистической разницы (0 в пределах доверительного интервала), а конверсия в пользователя, который вернулся в игру через 7 дней после установки имеет статистически значимое различие. Средняя конверсия в такого пользователя в версии gate_30 составляет 19.0%, а в версии gate_40 составляет 18.2%.

Статистическую значимость различий между количеством сыгранных раундов проверим с помощью критерия Манна-Уитни.

In [213]:
print(f'pvalue_1 = {stats.mannwhitneyu(game_rounds_g30, game_rounds_g40)[1]}')


pvalue_1 = 0.05020880772044255


In [214]:
game_rounds_g30.mean()


52.45626398210291

In [215]:
game_rounds_g40.mean()


51.29877552814966

Рассчитанное значение p-value > 0.05, т. е. выявлены статистически значимые различия между выборками. Среднее количество сыгранных раундов в версии gate_30 составляет 52,46 на игрока, в версии gate_40 составляет 51,30 на игрока.

# Выводы
В ходе анализа проведенного A/B тестирования выявлено следующее:
1. Статистически значимой разницы в проценте вернувшихся в игру игроков на следующий день после установки не выявлено.
2. Выявлена статистически значимая разница в проценте вернувшихся игроков в игру через неделю. В версии gate_30 таких  игроков больше (19,0% против 18,2%).
3. Выявлена статистически значимая разница в среднем количестве раундов, сыгранных игроками. В версии gate_30 этот показатель выше (52,46 против 51,30).

# Рекомендации
Рассчитанное значение p-value для выборки количества сыгранных раундов незначительно, но превышает пороговое значение 0.05, причем версия gate_40 показывает чуть худшие результаты, чем версия gate_30. Используя для сравнения только этот показатель, однозначно утверждать, что версия gate_40 ухудшила пользовательский опыт не можем. Однако, статистически значимая разница зафиксирована и в количестве вернувшихся в игру игроков через неделю после установки. И по этому показателю версия gate_40 также уступает версии gate_30, хотя и не критично.

Учитывая это, рекомендуем использовать для дальнейшего развития версию gate_30. Показатели версии gate_40 хоть и незначительно, но уступают версии gate_30.